## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [16]:
from collections import Counter

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [10]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [11]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [12]:
len(sentences_ted)

266694

In [13]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [41]:
counter = Counter()
counts_ted_top1000 =[]
words_top_ted =[]

for sent in sentences_ted:
    for word in sent:
        counter[word] +=1
count_dict  = counter.most_common(1000)
for item in count_dict: 
    k,v = item
    counts_ted_top1000.append(v)
    words_top_ted.append(k)
print(counts_ted_top1000[:10])

[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923]


Plot distribution of top-1000 words

In [30]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [32]:
from gensim.models import Word2Vec

In [35]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=20, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [52]:
model_ted.most_similar(positive=['woman', 'king'], negative=['man'])


[('queen', 0.763699471950531),
 ('french', 0.7622603178024292),
 ('president', 0.7551910877227783),
 ('charles', 0.7510802149772644),
 ('james', 0.7499485015869141),
 ('luther', 0.7474262118339539),
 ('martin', 0.7370063066482544),
 ('named', 0.731828510761261),
 ('poet', 0.7290182113647461),
 ('german', 0.7171764373779297)]

In [51]:
model_ted.most_similar("man")


[('woman', 0.8372774720191956),
 ('guy', 0.8022753000259399),
 ('gentleman', 0.7573291063308716),
 ('soldier', 0.7568672895431519),
 ('lady', 0.7398322820663452),
 ('girl', 0.7279258966445923),
 ('boy', 0.7156901359558105),
 ('kid', 0.6837275624275208),
 ('david', 0.6659721732139587),
 ('rabbi', 0.6647383570671082)]

In [50]:
man = model_ted['man']
women = model_ted['women']
d = np.dot(man,women)/(np.linalg.norm(man)*np.linalg.norm(women))
print(d)
print(model_ted.similarity('man','women'))

0.400974
0.400974246451


In [45]:
model_ted.most_similar("boat")

[('plane', 0.8177670240402222),
 ('floor', 0.7829046249389648),
 ('ship', 0.7708150744438171),
 ('truck', 0.767514705657959),
 ('road', 0.7561843991279602),
 ('chair', 0.745393693447113),
 ('wall', 0.7453821301460266),
 ('tent', 0.7447665929794312),
 ('roof', 0.736034095287323),
 ('corner', 0.7335935235023499)]

In [39]:
len(model_ted.vocab)

9501

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [42]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [43]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [44]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [53]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [54]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [55]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [56]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [70]:
print(sentences_wiki[:10])
sentence_wiki = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentence_wiki.append(tokens)
print(sentence_wiki[:10])

['  pm   marking the end of the first night of transmission for nbn       in newcastle ', ' the key location used was the oil refinery at shell haven in stanford     le     hope   essex   on the thames estuary   which represented the secret winnerden flats complex ', ' in june        abbe enlisted in the imperial japanese army   and was subsequently posted to a garrison in manchuria ', '       audio       ', '      points on the floor       ', ' wallen operated the roadhouse through the     s and early     s   although in later years guests were by invitation only ', ' empire named   no tv and no beer make homer go crazy   the sixth best film parody in the show  s history ', ' the singer later proceeded to put rough vocals on those tracks   and also brought in his own material for the band members to work on ', ' cena attempted to perform the fu   but khali blocked it with his elbow ', '    feet      ']
[['pm', 'marking', 'the', 'end', 'of', 'the', 'first', 'night', 'of', 'transmission

In [76]:
print(sentence_wiki[:10])

[['pm', 'marking', 'the', 'end', 'of', 'the', 'first', 'night', 'of', 'transmission', 'for', 'nbn', 'in', 'newcastle'], ['the', 'key', 'location', 'used', 'was', 'the', 'oil', 'refinery', 'at', 'shell', 'haven', 'in', 'stanford', 'le', 'hope', 'essex', 'on', 'the', 'thames', 'estuary', 'which', 'represented', 'the', 'secret', 'winnerden', 'flats', 'complex'], ['in', 'june', 'abbe', 'enlisted', 'in', 'the', 'imperial', 'japanese', 'army', 'and', 'was', 'subsequently', 'posted', 'to', 'a', 'garrison', 'in', 'manchuria'], ['audio'], ['points', 'on', 'the', 'floor'], ['wallen', 'operated', 'the', 'roadhouse', 'through', 'the', 's', 'and', 'early', 's', 'although', 'in', 'later', 'years', 'guests', 'were', 'by', 'invitation', 'only'], ['empire', 'named', 'no', 'tv', 'and', 'no', 'beer', 'make', 'homer', 'go', 'crazy', 'the', 'sixth', 'best', 'film', 'parody', 'in', 'the', 'show', 's', 'history'], ['the', 'singer', 'later', 'proceeded', 'to', 'put', 'rough', 'vocals', 'on', 'those', 'tracks'

In [78]:
counter = Counter()
counts_wiki_top1000 =[]
words_top_wiki =[]

for sent in sentence_wiki:
    for word in sent:
        counter[word] +=1
count_dict  = counter.most_common(1000)
for item in count_dict: 
    k,v = item
    counts_wiki_top1000.append(v)
    words_top_wiki.append(k)
print(counts_wiki_top1000[:10])
print(words_top_wiki[:10])

[1287766, 547842, 500106, 434780, 399025, 358161, 215092, 164344, 153531, 145026]
['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']


In [79]:
count_dict

[('the', 1287766),
 ('of', 547842),
 ('and', 500106),
 ('in', 434780),
 ('to', 399025),
 ('a', 358161),
 ('was', 215092),
 ('s', 164344),
 ('on', 153531),
 ('as', 145026),
 ('for', 140112),
 ('that', 138131),
 ('with', 129924),
 ('by', 126145),
 ('is', 103394),
 ('his', 92980),
 ('at', 92347),
 ('he', 91974),
 ('from', 89276),
 ('it', 88722),
 ('were', 70858),
 ('an', 64360),
 ('had', 58213),
 ('which', 55521),
 ('be', 47668),
 ('this', 43731),
 ('but', 42301),
 ('are', 41049),
 ('first', 40893),
 ('not', 39685),
 ('their', 38789),
 ('after', 38527),
 ('one', 37876),
 ('her', 37423),
 ('also', 36975),
 ('its', 36713),
 ('they', 34974),
 ('two', 34024),
 ('or', 33601),
 ('have', 32521),
 ('who', 32023),
 ('new', 30771),
 ('been', 30464),
 ('has', 30389),
 ('she', 29372),
 ('when', 27265),
 ('time', 26472),
 ('during', 26351),
 ('other', 25805),
 ('would', 25189),
 ('into', 24792),
 ('all', 24399),
 ('i', 23643),
 ('more', 22652),
 ('over', 21316),
 ('while', 21252),
 ('him', 20092),
 ('

In [83]:
model_wiki = Word2Vec(sentence_wiki, size=100, window=5, min_count=100, workers=4)

#### t-SNE visualization

In [84]:
len(model_wiki.vocab)

11666

In [85]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [86]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)